In [6]:
import numpy as np
from scipy.io import loadmat
from scipy import sparse

from sklearn.datasets import make_moons
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from sklearn.metrics import homogeneity_score, completeness_score,v_measure_score

import matplotlib.pyplot as plt


In [5]:
dataset = loadmat('coil20.mat')
label = np.array([dataset['Y'][i][0] - 1 for i in range(len(dataset['Y']))])
data = dataset['X']
n_class = len(np.unique(label))

In [7]:
n_neighbors = 10
knn_dist_graph = kneighbors_graph(X=data,
                                 n_neighbors=n_neighbors,
                                 mode='distance',
                                 metric='euclidean',
                                 n_jobs=6)

In [8]:
knn_dist_graph.todense()[0][:,:50]

matrix([[0.        , 1.26885262, 2.66702216, 3.67019378, 4.72927933,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]])

In [9]:
sigma = 1
similarity_graph = sparse.csr_matrix(knn_dist_graph.shape)
nonzeroindices = knn_dist_graph.nonzero()

similarity_graph[nonzeroindices] = np.exp(-np.asarray(knn_dist_graph[nonzeroindices])**2 / 2.0 * sigma**2)

similarity_graph = 0.5 * (similarity_graph + similarity_graph.T)
similarity_graph.todense()[0][:,:50]

C:\Users\yc000\miniconda3\envs\pm3env\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


matrix([[0.00000000e+00, 4.47090841e-01, 2.85384323e-02, 1.18838361e-03,
         1.39080678e-05, 1.89489814e-08, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00]])

In [10]:
degree_matrix = similarity_graph.sum(axis=1)
diagonal_matrix = np.diag(np.asarray(degree_matrix).reshape(len(data),))

L =  diagonal_matrix - similarity_graph

In [11]:
L

matrix([[ 0.87646982, -0.44709084, -0.02853843, ...,  0.        ,
          0.        ,  0.        ],
        [-0.44709084,  0.71876647, -0.20767633, ...,  0.        ,
          0.        ,  0.        ],
        [-0.02853843, -0.20767633,  0.67805609, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.48957492,
         -0.15864191, -0.0360337 ],
        [ 0.        ,  0.        ,  0.        , ..., -0.15864191,
          0.31722138, -0.08538867],
        [ 0.        ,  0.        ,  0.        , ..., -0.0360337 ,
         -0.08538867,  0.58968072]])